In [2]:
import torch

class MultivariateHawkesProcess:
    def __init__(self, mu, alpha, beta, dt_max=0.1, max_time=10.0):
        """
        Initializes the multivariate Hawkes process.

        Parameters:
        - mu: Base intensities (vector of shape [num_dimensions])
        - alpha: Influence matrix (matrix of shape [num_dimensions, num_dimensions])
        - beta: Decay rates (vector of shape [num_dimensions])
        - dt_max: Maximum time step to simulate between events
        - max_time: Time horizon for the simulation
        """
        self.mu = mu  # base intensity, shape [num_dimensions]
        self.alpha = alpha  # influence matrix, shape [num_dimensions, num_dimensions]
        self.beta = beta  # decay rates, shape [num_dimensions]
        self.dt_max = dt_max  # maximum time step
        self.max_time = max_time  # time horizon

        self.num_dimensions = mu.shape[0]
        self.events = [[] for _ in range(self.num_dimensions)]  # List of event times for each dimension

    def simulate(self):
        current_time = 0.0
        intensities = self.mu.clone()  # initialize with base intensities

        while current_time < self.max_time:
            # Calculate total intensity for the next event
            total_intensity = intensities.sum()

            # Time to next event
            dt = torch.distributions.Exponential(total_intensity).sample().item()
            if current_time + dt > self.max_time:
                break
            current_time += dt

            # Choose which dimension the event happens in
            probs = intensities / total_intensity
            event_dimension = torch.multinomial(probs, 1).item()

            # Record the event
            self.events[event_dimension].append(current_time)

            # Update intensities based on the new event
            for i in range(self.num_dimensions):
                intensities[i] = self.mu[i] + torch.sum(
                    self.alpha[i, event_dimension] * torch.exp(-self.beta[i] * (current_time - torch.tensor(self.events[event_dimension])))
                )

        return self.events



In [3]:
# Example Usage

# Define parameters for a 3-dimensional Hawkes process
mu = torch.tensor([0.2, 0.15, 0.1])  # base intensities
alpha = torch.tensor([[0.5, 0.3, 0.1],
                      [0.2, 0.4, 0.3],
                      [0.1, 0.2, 0.5]])  # influence matrix
beta = torch.tensor([1.0, 1.5, 1.2])  # decay rates

# Instantiate the process
hawkes_process = MultivariateHawkesProcess(mu, alpha, beta, max_time=20.0)

# Simulate events
events = hawkes_process.simulate()

# Print results
for i, event_times in enumerate(events):
    print(f"Dimension {i+1}: {event_times}")


Dimension 1: [0.7813360095024109, 2.3544456362724304, 5.6284236535429955, 6.894859157502651, 9.412772871553898, 10.238099433481693, 10.962426699697971, 11.40073136240244, 11.660657294094563, 12.615419931709766, 12.672868099063635, 13.93748315796256, 14.035546038299799, 15.539317939430475, 15.955983493477106, 16.775773379951715, 16.901649449020624, 17.16190940514207, 17.64105201140046, 18.36843717470765, 18.369470797711983]
Dimension 2: [0.09641486406326294, 3.6133491471409798, 7.26455333083868, 18.718938045436516, 19.83251779549755, 19.95415820949711]
Dimension 3: [3.5011715292930603, 4.71411819010973, 4.960074864327908, 8.249268256127834, 8.396996088325977, 8.854772008955479]
